# EE4065 Final Project - Q4 Master Notebook

This notebook handles data loading, model training (MobileNet, ResNet, SqueezeNet), and conversion to TFLite Int8 for ESP32.


In [ ]:
!nvidia-smi


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import os

# Create directory in Drive
TARGET_DIR = '/content/drive/MyDrive/EE4065_Q4'
os.makedirs(TARGET_DIR, exist_ok=True)
%cd {TARGET_DIR}


In [ ]:
%%writefile mobilenetv1.py
# /*---------------------------------------------------------------------------------------------
#  * Copyright 2015 The TensorFlow Authors.
#  * Copyright (c) 2022-2023 STMicroelectronics.
#  * All rights reserved.
#  *
#  * This software is licensed under terms that can be found in the LICENSE file in
#  * the root directory of this software component.
#  * If no LICENSE file comes with this software, it is provided AS-IS.
#  *--------------------------------------------------------------------------------------------*/

import tensorflow as tf
from tensorflow import keras
from keras import layers
from typing import Tuple


def _depthwise_conv_block(inputs, filters, alpha, depth_multiplier=1, strides=(1, 1), block_id=1):
    """Adds a depthwise convolution block.

    This function defines a depthwise convolution block for use in a mobile
    architecture. The block consists of a depthwise convolution, followed by
    a pointwise convolution, with batch normalization and ReLU6 activations.

    Args:
        inputs (tensor): Input tensor.
        filters (int): Number of filters for the pointwise convolution.
        alpha (float): Width multiplier for the number of filters.
        depth_multiplier (int, optional): Depth multiplier for the depthwise convolution.
        strides (tuple, optional): Strides for the depthwise convolution.
        block_id (int, optional): Block identifier.

    Returns:
        tensor: Output tensor.

    """
    # Calculate the number of filters for the pointwise convolution.
    pointwise_conv_filters = int(filters * alpha)

    if strides == (1, 1):
        x = inputs
    else:
        # If the strides are not (1, 1), pad the input tensor to maintain the same output size.
        x = layers.ZeroPadding2D(((0, 1), (0, 1)))(inputs)
    # Perform the depthwise convolution.
    x = layers.DepthwiseConv2D(kernel_size=(3, 3), padding="same" if strides == (1, 1) else "valid",
                               depth_multiplier=depth_multiplier, strides=strides, use_bias=False,
                               name="conv_dw_%d" % block_id, )(x)
    x = layers.BatchNormalization(name="conv_dw_%d_bn" % block_id)(x)
    x = layers.ReLU(6.0, name="conv_dw_%d_relu" % block_id)(x)

    # Perform the pointwise convolution.
    x = layers.Conv2D(pointwise_conv_filters, kernel_size=(1, 1), padding="same", use_bias=False, strides=(1, 1),
                      name="conv_pw_%d" % block_id)(x)
    x = layers.BatchNormalization(name="conv_pw_%d_bn" % block_id)(x)
    x = layers.ReLU(6.0, name="conv_pw_%d_relu" % block_id)(x)

    return x


def _get_scratch_model(input_shape: tuple = None, alpha: float = None, num_classes: int = None, 
                      dropout: float = None) -> tf.keras.Model:
    """Get a MobileNet V1 model from scratch.

    This function defines a MobileNet V1 model from scratch using depthwise
    convolution blocks.

    Args:
        input_shape (tuple): Shape of the input tensor.
        num_classes (int): Number of output classes.
        alpha (float): Width multiplier for the number of filters.
        dropout (float, optional): Dropout rate.

    Returns:
        model: MobileNet V1 model.

    """
    # Define the input tensor.
    inputs = keras.Input(shape=input_shape)

    # First convolution block.
    first_block_filters = int(32 * alpha)
    x = layers.Conv2D(first_block_filters, kernel_size=3, strides=(2, 2), padding='same', use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU(6.)(x)

    # Depthwise convolution blocks.
    x = _depthwise_conv_block(x, filters=64, alpha=alpha, strides=(1, 1), block_id=1)

    x = _depthwise_conv_block(x, filters=128, alpha=alpha, strides=(2, 2), block_id=2)
    x = _depthwise_conv_block(x, filters=128, alpha=alpha, strides=(1, 1), block_id=3)

    x = _depthwise_conv_block(x, filters=256, alpha=alpha, strides=(2, 2), block_id=4)
    x = _depthwise_conv_block(x, filters=256, alpha=alpha, strides=(1, 1), block_id=5)

    x = _depthwise_conv_block(x, filters=512, alpha=alpha, strides=(2, 2), block_id=6)

    x = _depthwise_conv_block(x, filters=512, alpha=alpha, strides=(1, 1), block_id=7)
    x = _depthwise_conv_block(x, filters=512, alpha=alpha, strides=(1, 1), block_id=8)
    x = _depthwise_conv_block(x, filters=512, alpha=alpha, strides=(1, 1), block_id=9)
    x = _depthwise_conv_block(x, filters=512, alpha=alpha, strides=(1, 1), block_id=10)
    x = _depthwise_conv_block(x, filters=512, alpha=alpha, strides=(1, 1), block_id=11)

    x = _depthwise_conv_block(x, filters=1024, alpha=alpha, strides=(2, 2), block_id=12)
    x = _depthwise_conv_block(x, filters=1024, alpha=alpha, strides=(1, 1), block_id=13)

    # Global average pooling and output layer.
    x = keras.layers.GlobalAveragePooling2D()(x)
    if dropout:
        x = keras.layers.Dropout(dropout)(x)
    if num_classes > 2:
        outputs = keras.layers.Dense(num_classes, activation="softmax")(x)
    else:
        outputs = keras.layers.Dense(1, activation="sigmoid")(x)

    # Define the model.
    model = keras.Model(inputs=inputs, outputs=outputs, name="mobilenet_v1_alpha_{}".format(alpha))

    return model


def _get_transfer_learning_model(input_shape: Tuple[int, int, int] = None, alpha: float = None,
                                num_classes: int = None, dropout: float = None,
                                weights: str = None) -> tf.keras.Model:
    """
    Creates a transfer learning model using the MobileNet architecture.

    Args:
        input_shape: A tuple representing the input shape of the model.
        dropout: A float representing the dropout rate of the model.
        alpha: A float representing the width multiplier of the MobileNet architecture.
        num_classes (int): Number of output classes. Default is None.
        weights (str, optional): The pre-trained weights to use. Either "imagenet" or None. Defaults to None.
    Returns:
        A Keras model object with the MobileNet architecture as the backbone and a randomly initialized head.
    """
    # Create a randomly initialized model
    random_model = _get_scratch_model(input_shape=input_shape, num_classes=num_classes, alpha=alpha, dropout=dropout)

    # Check if input shape is valid for MobileNet architecture
    if input_shape[0] in [224, 192, 160, 128]:
        input_shape = (input_shape[0], input_shape[1], input_shape[2])
        # Use MobileNet architecture with specified input shape
        backbone = tf.keras.applications.MobileNet(input_shape, input_tensor=random_model.inputs[0],
                                                   alpha=alpha, weights=weights, include_top=False)
    else:
        # Use default MobileNet architecture
        backbone = tf.keras.applications.MobileNet(input_tensor=random_model.inputs[0],
                                                   alpha=alpha, weights=weights, include_top=False)

    # Copy weights from MobileNet backbone to randomly initialized model
    for i, layer in enumerate(backbone.layers):
        random_model.layers[i].set_weights(layer.get_weights())

    # Return the transfer learning model
    return random_model


def get_mobilenetv1(input_shape: tuple, alpha: float = None, 
                     num_classes: int = None, dropout: float = None,
                     pretrained_weights: str = "imagenet") -> tf.keras.Model:
    """
    Returns a MobileNetV1 model with a custom classifier.

    Args:
        input_shape (tuple): The shape of the input tensor.
        alpha (float, optional): The width multiplier for the MobileNetV1 backbone. Defaults to None.
        dropout (float, optional): The dropout rate for the MobileNetV1 backbone. Defaults to None.
        num_classes (int, optional): The number of output classes. Defaults to None.
        weights (str, optional): The pre-trained weights to use. Either "imagenet" or None.
                                 Defaults to "imagenet".

    Returns:
        tf.keras.Model: The MobileNetV1 model with a custom classifier.
    """
    
    if pretrained_weights:
        model = _get_transfer_learning_model(input_shape=input_shape, alpha=alpha,
                                            num_classes=num_classes, dropout=dropout,
                                            weights=pretrained_weights)
    else:
        model = _get_scratch_model(input_shape=input_shape, alpha=alpha,
                                  num_classes=num_classes, dropout=dropout)

    return model


In [ ]:
%%writefile resnetv1.py
# /*---------------------------------------------------------------------------------------------
#  * Copyright (c) 2022 STMicroelectronics.
#  * All rights reserved.
#  * This software is licensed under terms that can be found in the LICENSE file in
#  * the root directory of this software component.
#  * If no LICENSE file comes with this software, it is provided AS-IS.
#  *--------------------------------------------------------------------------------------------*/

from keras.models import Model
from keras.regularizers import l2
from tensorflow import keras
from keras import layers
from typing import Tuple


def _resnet_layer(inputs: layers.Input, num_filters: int = 16, kernel_size: int = 3, strides: int = 1,
                 activation: str = 'relu', batch_normalization: bool = True,
                 conv_first: bool = True) -> layers.Activation:
    """
    2D Convolution-Batch Normalization-Activation stack builder for ResNet models.

    Args:
        inputs: Input tensor from input image or previous layer.
        num_filters: Conv2D number of filters.
        kernel_size: Conv2D square kernel dimensions.
        strides: Conv2D square stride dimensions.
        activation: Activation name.
        batch_normalization: Whether to include batch normalization.
        conv_first: Conv-BN-Activation (True) or BN-Activation-Conv (False).

    Returns:
        A tensor as input to the next layer.
    """
    conv = layers.Conv2D(num_filters,
                         kernel_size=kernel_size,
                         strides=strides,
                         padding='same',
                         kernel_initializer='he_normal',
                         kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = layers.BatchNormalization()(x)
        if activation is not None:
            x = layers.Activation(activation)(x)
    else:
        if batch_normalization:
            x = layers.BatchNormalization()(x)
        if activation is not None:
            x = layers.Activation(activation)(x)
        x = conv(x)
    return x


def get_resnetv1(num_classes: int = None, input_shape: Tuple[int, int, int] = None,
                 depth: int = None, dropout: float = None) -> keras.Model:
    """
    ResNet Version 1 Model builder.

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU. Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (down-sampled)
    by a convolutional layer with strides=2, while the number of filters is doubled.
    Within each stage, the layers have the same number filters and the same number of filters.

    Args:
        num_classes: Number of classes in the dataset.
        input_shape: Shape of the input tensor.
        depth: Depth of the ResNet model.
        dropout: Dropout rate to be applied to the fully connected layer.

    Returns:
        A Keras model instance.
    """
    if (depth - 2) % 6 != 0:
        raise ValueError("Depth should be 6n+2.")

    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = keras.Input(shape=input_shape)
    x = _resnet_layer(inputs=inputs)

    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # down sample
            y = _resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = _resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match changed dims
                x = _resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = layers.add([x, y])
            x = layers.Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = layers.AveragePooling2D(pool_size=8)(x)
    x = layers.Flatten()(x)
    if dropout:
        x = layers.Dropout(dropout)(x)

    if num_classes > 2:
        outputs = keras.layers.Dense(num_classes, activation="softmax", kernel_initializer='he_normal')(x)
    else:
        outputs = layers.Dense(1, activation="sigmoid")(x)

    # Instantiate model.
    model = keras.Model(inputs=inputs, outputs=outputs, name=f"resnet_v1_depth_{depth}")
    return model


In [ ]:
%%writefile squeezenetv11.py
# /*---------------------------------------------------------------------------------------------
#  * Copyright (c) 2016 Refikcanmalli
#  * Copyright (c) 2022 STMicroelectronics.
#  * All rights reserved.
#  * This software is licensed under terms that can be found in the LICENSE file in
#  * the root directory of this software component.
#  * If no LICENSE file comes with this software, it is provided AS-IS.
#  *--------------------------------------------------------------------------------------------*/
from tensorflow import keras
#from tensorflow.keras import layers
from keras import layers
from typing import Tuple


def _fire_module(x: keras.layers.Layer, fire_id: int, squeeze: int = 16, expand: int = 64) -> keras.layers.Layer:
    """
    Fire module for the SqueezeNet model.
    Implements expand layer, which has a mix of 1x1 and 3x3 filters,
    by using two conv layers concatenated in the channel dimension.
    """
    # Define some constants
    sq1x1 = "squeeze1x1"
    exp1x1 = "expand1x1"
    exp3x3 = "expand3x3"
    relu = "relu_"
    bn = "bn_"

    s_id = 'fire' + str(fire_id) + '_'

    # Get the channel axis
    if keras.backend.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = 3

    # Squeeze layer
    x = layers.Conv2D(squeeze, (1, 1), padding='valid', name=s_id + sq1x1)(x)
    x = layers.BatchNormalization(axis=channel_axis, name=s_id + bn + sq1x1)(x)
    x = layers.Activation('relu', name=s_id + relu + sq1x1)(x)

    # Expand layer
    left = layers.Conv2D(expand, (1, 1), padding='valid', name=s_id + exp1x1)(x)
    left = layers.BatchNormalization(axis=channel_axis, name=s_id + bn + exp1x1)(left)
    left = layers.Activation('relu', name=s_id + relu + exp1x1)(left)

    right = layers.Conv2D(expand, (3, 3), padding='same', name=s_id + exp3x3)(x)
    right = layers.BatchNormalization(axis=channel_axis, name=s_id + bn + exp3x3)(right)
    right = layers.Activation('relu', name=s_id + relu + exp3x3)(right)

    x = layers.concatenate([left, right], axis=channel_axis, name=s_id + 'concat')

    return x


def get_squeezenetv11(num_classes: int = None, input_shape: Tuple[int, int, int] = None,
                      dropout: float = None) -> keras.Model:
    """
    Returns a SqueezeNet model with the specified number of output classes.
    """
    # Define the input tensor
    input_image = layers.Input(input_shape)

    # First convolutional layer
    x = layers.Conv2D(64, (3, 3), strides=(2, 2), padding='valid', name='conv1')(input_image)
    x = layers.BatchNormalization(name='bn_conv1')(x)
    x = layers.Activation('relu', name='relu_conv1')(x)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool1')(x)

    # Fire modules
    x = _fire_module(x, fire_id=2, squeeze=16, expand=64)
    x = _fire_module(x, fire_id=3, squeeze=16, expand=64)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool3')(x)

    x = _fire_module(x, fire_id=4, squeeze=32, expand=128)
    x = _fire_module(x, fire_id=5, squeeze=32, expand=128)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool5')(x)

    x = _fire_module(x, fire_id=6, squeeze=48, expand=192)
    x = _fire_module(x, fire_id=7, squeeze=48, expand=192)
    x = _fire_module(x, fire_id=8, squeeze=64, expand=256)
    x = _fire_module(x, fire_id=9, squeeze=64, expand=256)

    # Dropout layer
    if dropout:
        x = layers.Dropout(dropout, name='drop9')(x)

    # Final convolutional layer
    x = layers.Conv2D(num_classes, (1, 1), padding='valid', name='conv10')(x)
    
    # Global average pooling layer
    x = layers.GlobalAveragePooling2D()(x)

    # Softmax activation layer
    output = layers.Activation('softmax', name='loss')(x)

    # Create the model
    model = keras.Model(input_image, output, name='SqueezeNet_v1.1')

    return model


In [ ]:
%%writefile train_models.py
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from mobilenetv1 import get_mobilenetv1
from resnetv1 import get_resnetv1
from squeezenetv11 import get_squeezenetv11

# Configuration
IMG_SIZE = 64
BATCH_SIZE = 32
EPOCHS = 20
NUM_CLASSES = 10
MODELS_DIR = "models"
PLOTS_DIR = "plots"

def load_data():
    print("Loading MNIST dataset...")
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

    # Normalize to [0, 1]
    x_train = x_train.astype("float32") / 255.0
    x_test = x_test.astype("float32") / 255.0

    # Add channel dimension
    x_train = np.expand_dims(x_train, axis=-1)
    x_test = np.expand_dims(x_test, axis=-1)

    return (x_train, y_train), (x_test, y_test)

def create_datasets(x_train, y_train, x_test, y_test):
    def preprocess(image, label):
        # Resize to 64x64
        image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
        return image, label

    train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_ds = train_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    train_ds = train_ds.shuffle(10000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    test_ds = test_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    return train_ds, test_ds

def save_plots(history, y_true, y_pred_classes, model_name):
    if not os.path.exists(PLOTS_DIR):
        os.makedirs(PLOTS_DIR)

    # 1. Accuracy & Loss
    plt.figure(figsize=(12, 5))
    
    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.title(f'{model_name} Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title(f'{model_name} Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.savefig(os.path.join(PLOTS_DIR, f"{model_name}_metrics.png"))
    plt.close()

    # 2. Confusion Matrix
    cm = confusion_matrix(y_true, y_pred_classes)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=range(10), yticklabels=range(10))
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig(os.path.join(PLOTS_DIR, f"{model_name}_confusion_matrix.png"))
    plt.close()

    # Print Classification Report
    print(f"\n--- Classification Report for {model_name} ---")
    print(classification_report(y_true, y_pred_classes, digits=4))

def train_and_evaluate(model, train_ds, test_ds, y_test, model_name, input_shape):
    print(f"\n========================================")
    print(f"Training {model_name}...")
    print(f"========================================")
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    history = model.fit(train_ds, epochs=EPOCHS, validation_data=test_ds)
    
    # Save Model
    model.save(os.path.join(MODELS_DIR, f"{model_name}.keras"))
    print(f"{model_name} saved.")

    # Evaluate on Test Set
    print(f"Evaluating {model_name}...")
    # Predict
    y_pred_probs = model.predict(test_ds)
    y_pred_classes = np.argmax(y_pred_probs, axis=1)

    # Generate Plots
    save_plots(history, y_test, y_pred_classes, model_name)

def main():
    if not os.path.exists(MODELS_DIR):
        os.makedirs(MODELS_DIR)

    (x_train, y_train), (x_test, y_test) = load_data()
    train_ds, test_ds = create_datasets(x_train, y_train, x_test, y_test)
    input_shape = (IMG_SIZE, IMG_SIZE, 1)

    # 1. MobileNet
    mobilenet = get_mobilenetv1(input_shape=input_shape, alpha=0.25, num_classes=NUM_CLASSES, pretrained_weights=None)
    train_and_evaluate(mobilenet, train_ds, test_ds, y_test, "mobilenetv1", input_shape)

    # 2. ResNet
    # ResNet20
    resnet = get_resnetv1(input_shape=input_shape, depth=20, num_classes=NUM_CLASSES)
    train_and_evaluate(resnet, train_ds, test_ds, y_test, "resnetv1", input_shape)

    # 3. SqueezeNet
    squeezenet = get_squeezenetv11(input_shape=input_shape, num_classes=NUM_CLASSES)
    train_and_evaluate(squeezenet, train_ds, test_ds, y_test, "squeezenetv11", input_shape)

if __name__ == "__main__":
    main()


In [ ]:
%%writefile convert_models.py
import tensorflow as tf
import numpy as np
import os

IMG_SIZE = 64
MODELS_DIR = "models"
HEADERS_DIR = "headers"

def load_data_for_calibration():
    print("Loading MNIST dataset for calibration...")
    (x_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
    
    # Just take a subset for calibration
    x_train = x_train[:1000]
    
    # Normalize to [0, 1]
    x_train = x_train.astype("float32") / 255.0
    
    # Add channel dimension
    x_train = np.expand_dims(x_train, axis=-1)
    resized_images = []
    for img in x_train:
        # tf.image.resize expects 3D or 4D tensor
        img_tensor = tf.convert_to_tensor(img)
        img_resized = tf.image.resize(img_tensor, [IMG_SIZE, IMG_SIZE])
        resized_images.append(img_resized.numpy())
    
    return np.array(resized_images)

def convert_to_tflite_int8(model_path, calibration_data):
    print(f"Converting {model_path} to TFLite Int8...")
    
    # Load Keras model
    try:
        model = tf.keras.models.load_model(model_path)
    except Exception as e:
        print(f"Error loading model {model_path}: {e}")
        return None

    # Create converter
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    
    # Int8 Quantization settings
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    
    def rep_dataset():
        for i in range(len(calibration_data)):
            # Yield: [1, 64, 64, 1]
            yield [np.expand_dims(calibration_data[i], axis=0)]
            
    converter.representative_dataset = rep_dataset
    
    # Ensure full integer quantization
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.int8
    converter.inference_output_type = tf.int8
    
    tflite_model = converter.convert()
    return tflite_model

def convert_to_c_array(tflite_model, model_name):
    # Hex dump
    hex_array = []
    for byte in tflite_model:
        hex_array.append(f"0x{byte:02x}")
    
    # Format C code
    c_code = f"// Auto-generated C header for {model_name}\n"
    c_code += f"#ifndef {model_name.upper()}_MODEL_H\n"
    c_code += f"#define {model_name.upper()}_MODEL_H\n\n"
    c_code += f"const unsigned char {model_name}_model[] = {{\n"
    
    # Group by 12 bytes per line for readability
    for i in range(0, len(hex_array), 12):
        line = ", ".join(hex_array[i:i+12])
        c_code += f"  {line},\n"
        
    c_code += "};\n\n"
    c_code += f"const unsigned int {model_name}_model_len = {len(tflite_model)};\n\n"
    c_code += "#endif\n"
    
    return c_code

def main():
    if not os.path.exists(HEADERS_DIR):
        os.makedirs(HEADERS_DIR)
        
    calibration_data = load_data_for_calibration()
    
    model_files = [
        "mobilenetv1.keras",
        "resnetv1.keras",
        "squeezenetv11.keras"
    ]
    
    for filename in model_files:
        model_name = filename.split('.')[0]
        model_path = os.path.join(MODELS_DIR, filename)
        
        if not os.path.exists(model_path):
            print(f"Model file not found: {model_path}. Skipping.")
            continue
            
        tflite_model = convert_to_tflite_int8(model_path, calibration_data)
        
        if tflite_model:
            # Save .tflite file
            tflite_path = os.path.join(HEADERS_DIR, f"{model_name}.tflite")
            with open(tflite_path, "wb") as f:
                f.write(tflite_model)
            print(f"Saved {tflite_path}")
                
            header_name = model_name.replace("v1", "").replace("v11", "") + "_model" 
            
            c_code = convert_to_c_array(tflite_model, header_name)
            
            header_path = os.path.join(HEADERS_DIR, f"{header_name}.h")
            with open(header_path, "w") as f:
                f.write(c_code)
            print(f"Saved {header_path}")
            
            # Verify first few lines
            if "mobilenet" in header_name:
                print(f"\n--- Verification: First 5 lines of {header_path} ---")
                print("\n".join(c_code.splitlines()[:5]))
                print("----------------------------------------------------\n")

if __name__ == "__main__":
    main()


In [ ]:
!python train_models.py


In [ ]:
!python convert_models.py


In [ ]:
!ls -l headers/


In [ ]:
!head -n 20 headers/mobilenet_model.h
